In [1]:
import torch

In [2]:
model = torch.load('/home/marc/PhD/PanRBPNet/models/model.LikeBasenji2.pt')
model.eval()

/home/marc/.miniconda/envs/parnet/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Seed set to 42
2024-04-20 12:19:43.823549: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-20 12:19:43.823584: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-20 12:19:43.824785: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-20 12:19:43.832035: I tensorflow/core/platform/cpu_feature_guard.cc:182] T

LikeBasenji2(
  (stem): Sequential(
    (0): Conv1d(4, 384, kernel_size=(11,), stride=(1,), padding=same)
    (1): BatchNorm1d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): GELU(approximate='none')
  )
  (conv_tower): Sequential(
    (0): LikeBasenji2ConvBlock(
      (conv1d): Conv1d(384, 768, kernel_size=(5,), stride=(1,), padding=same)
      (batch_norm): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): GELU(approximate='none')
    )
    (1): LikeBasenji2ConvBlock(
      (conv1d): Conv1d(768, 768, kernel_size=(5,), stride=(1,), padding=same)
      (batch_norm): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): GELU(approximate='none')
    )
    (2): LikeBasenji2ConvBlock(
      (conv1d): Conv1d(768, 768, kernel_size=(5,), stride=(1,), padding=same)
      (batch_norm): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): GELU

In [3]:
model.head = torch.nn.Identity()

In [4]:
from parnet.utils import sequence_to_onehot

seq = sequence_to_onehot('ACGT'*100).unsqueeze(0).float()
seq.shape

torch.Size([1, 4, 400])

In [7]:
pred = model({'sequence': seq})
pred.shape

torch.Size([1, 960, 400])

In [13]:
class NoHeadModel(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.model.head = torch.nn.Identity()
        
    def forward(self, x):
        return self.model(x)
    
    def forward_mean_pool(self, x):
        return self.model(x).mean(dim=-1)

model_no_head = NoHeadModel(model)
pred = model_no_head.forward_mean_pool({'sequence': seq})
pred.shape

torch.Size([1, 960])

In [9]:
d = {'a': [1, 2, 3], 'b': [4, 5, 6]}

In [17]:
import pandas as pd

df = pd.DataFrame(d).T
df.columns = df.columns.map(lambda i: f'dim{i}')
df

,dim0,dim1,dim2
a,1,2,3
b,4,5,6
